In [1]:
# Setup

# Packages
import os
from localutils import localcube

# main variables
csv_model, json_model, manifest_basename = localcube.import_config()

In [2]:
# print(f"schematic manifest --config {config['paths']['schematic']} get")

In [3]:
# manifest_path = "C:/Users/nlee/Documents/Projects/schematic/schematic/tests/data/mock_manifests/example_biospecimen_test.csv"
# !schematic model --config config['paths']['schematic'] /
#     validate --manifest_path  manifest_path/
#     --data_type Biospecimen

In [ ]:
import json


def get_mainfest_names(json_object):
    """Helper function for create dca config. Relies on having template in the properties column of templates"""
    # Manifest names in data model
    manifest_names_extracted = []

    for i in json_object["@graph"]:
        if i["@id"] == "bts:template":
            manifest_names_extracted += [
                t["@id"].replace("bts:", "") for t in i["schema:domainIncludes"]
            ]

    # display names extracted
    manifest_display_names_extracted = []

    for i in json_object["@graph"]:
        if i["@id"].replace("bts:", "") in (manifest_names_extracted):
            manifest_display_names_extracted.append(i["sms:displayName"])

    # Create dictionary for lookup later
    manifest_name_relationships = dict(
        zip(manifest_names_extracted, manifest_display_names_extracted)
    )

    return manifest_name_relationships

# Data modeling functions for manifest generation


def manifest_schema(k, v, t="file"):
    """Helper function for create dca config"""
    schema = {"display_name": v, "schema_name": k, "type": t}

    return schema


def create_dca_config(json_model):
    # look up dca config
    # if dca config exisits -> update schema version
    # else create dca config

    dca_template = {
        "manifest_schemas": [],
        "service_version": "v23.1.1",
        "schema_version": "v1.1",
    }

    records = ["FileMapToIndividual", "IndividualMetadata"]

    with open(json_model, "r") as jf:
        jo = json.load(jf)

    manifest_dict = get_mainfest_names(jo)

    for k, v in manifest_dict.items():
        if k in records:
            t = "records"
        else:
            t = "file"
        dca_template["manifest_schemas"].append(manifest_schema(k, v, t))

    json_formatted_str = json.dumps(dca_template, indent=2)

    print(json_formatted_str)

    with open(
        "../dca-template-config.json",
        "w",
    ) as f:
        f.write(json_formatted_str)

{'FileMapToIndividual': 'FileMapToIndividual',
 'IndividualMetadata': 'Individual Metadata'}

In [21]:
create_dca_config(json_model)

{
  "manifest_schemas": [
    {
      "display_name": "FileMapToIndividual",
      "schema_name": "FileMapToIndividual",
      "type": "records"
    },
    {
      "display_name": "Individual Metadata",
      "schema_name": "IndividualMetadata",
      "type": "records"
    }
  ],
  "service_version": "v23.1.1",
  "schema_version": "v1.1"
}
